<a href="https://colab.research.google.com/github/krishnannarayanaswamy/ptc-api/blob/main/PTC_Data_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cassio openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 782.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninsta

In [2]:
from getpass import getpass
ASTRA_DB_ID = input("ASTRA_DB_ID = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

desired_keyspace = input("ASTRA_DB_KEYSPACE (optional, can be left empty) = ")
if desired_keyspace:
    ASTRA_DB_KEYSPACE = desired_keyspace
else:
    ASTRA_DB_KEYSPACE = None

ASTRA_DB_ID = 132d3197-3812-4b8c-84b1-f69c9c4415ac
ASTRA_DB_APPLICATION_TOKEN = ··········
ASTRA_DB_KEYSPACE (optional, can be left empty) = products


In [3]:
import cassio

cassio.init(
    database_id=ASTRA_DB_ID,
    token=ASTRA_DB_APPLICATION_TOKEN,
    keyspace=ASTRA_DB_KEYSPACE,
)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(132390532913504) 132d3197-3812-4b8c-84b1-f69c9c4415ac-ap-south-1.db.astra.datastax.com:29042:79abf547-3d2e-4558-9c56-6624a3d328e5> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [4]:
import os
from langchain_community.embeddings import OpenAIEmbeddings
OPENAI_API_KEY = getpass("OPENAI_API_KEY = ")
#OPENAI_API_KEY = openai_api_key

#embe = OpenAIEmbeddings()

OPENAI_API_KEY = ··········


In [5]:
keyspace = ASTRA_DB_KEYSPACE
table = "ptc_inventory"
history_table = "chat_history"

cmd_create_products_table = f"""CREATE TABLE IF NOT EXISTS {keyspace}.{table}
(product_id int,
 item_sku text,
 item_name text,
 tags text,
 brand text,
 category text,
 unit_price float,
 short_description text,
 description text,
 product_url text,
 image text,
 text_embedding vector<float, 1536>,
 image_embedding vector<float, 1536>,
 PRIMARY KEY (product_id))"""

cmd_create_history_table = f"""CREATE TABLE IF NOT EXISTS {keyspace}.{history_table}
(id uuid,
 sessionid text,
 msgtime timestamp,
 userquery text,
 aianswer text,
 PRIMARY KEY (id))"""

cmd_create_vector_index = f"""CREATE CUSTOM INDEX IF NOT EXISTS text_embedding_index ON {keyspace}.{table}(text_embedding)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'similarity_function': 'dot_product' }};"""

cmd_create_vector_second_index = f"""CREATE CUSTOM INDEX IF NOT EXISTS image_embedding_index ON {keyspace}.{table}(image_embedding)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'similarity_function': 'dot_product' }};"""

cmd_create_vector_brand_index =  f"""
          CREATE CUSTOM INDEX IF NOT EXISTS brand_index ON {keyspace}.{table}(brand)
          USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'index_analyzer': '{{"tokenizer" : {{"name" : "standard"}},
            "filters" : [{{"name" : "porterstem"}}]}}'}};
          """

cmd_insert_product = f"""
                INSERT INTO {keyspace}.{table}
                (product_id, item_sku, item_name, tags, brand,category, unit_price, short_description, description, product_url, image, text_embedding)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """



In [6]:
s = cassio.config.resolve_session()

In [7]:
s.execute(cmd_create_products_table)
s.execute(cmd_create_history_table)
s.execute(cmd_create_vector_index)
s.execute(cmd_create_vector_second_index)
s.execute(cmd_create_vector_brand_index)

In [8]:
# More functions used on the code
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)
client.api_key = OPENAI_API_KEY

def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = client.embeddings.create(model=model, input=text)
    return response.data[0].embedding

In [11]:
import pandas as pd

data_file = 'Product_Website_Load.csv'
df = pd.read_csv(data_file)
print(df.columns)

Index(['product_id', 'item_sku', 'item_name', 'tags', 'brand', 'category',
       'unit_price', 'short_description', 'description', 'product_url',
       'image'],
      dtype='object')


In [12]:
count = 0
for index, row in df.iterrows():
  count += 1
  rawtext = f"Item SKU: {row.item_sku} Item Name: {row.item_name} Tags: {row.tags} Short Description: {row.short_description} Description: {row.description} Brand: {row.brand} Price: {row.unit_price} Category: {row.category}"
  #print(rawtext)
  openai_description_embedding = generate_embedding(rawtext)

  print(f""" id: {row.product_id} and {count} records inserted.""")

  s.execute(cmd_insert_product, (int(row.product_id), str(row.item_sku), str(row.item_name), str(row.tags), str(row.brand), str(row.category), float(row.unit_price), str(row.short_description),str(row.description), str(row.product_url), str(row.image), openai_description_embedding))

 id: 12 and 1 records inserted.
 id: 26 and 2 records inserted.
 id: 28 and 3 records inserted.
 id: 30 and 4 records inserted.
 id: 32 and 5 records inserted.
 id: 34 and 6 records inserted.
 id: 35 and 7 records inserted.
 id: 36 and 8 records inserted.
 id: 37 and 9 records inserted.
 id: 38 and 10 records inserted.
 id: 41 and 11 records inserted.
 id: 66 and 12 records inserted.
 id: 68 and 13 records inserted.
 id: 70 and 14 records inserted.
 id: 72 and 15 records inserted.
 id: 73 and 16 records inserted.
 id: 81 and 17 records inserted.
 id: 82 and 18 records inserted.
 id: 83 and 19 records inserted.
 id: 84 and 20 records inserted.
 id: 85 and 21 records inserted.
 id: 86 and 22 records inserted.
 id: 87 and 23 records inserted.
 id: 89 and 24 records inserted.
 id: 90 and 25 records inserted.
 id: 92 and 26 records inserted.
 id: 94 and 27 records inserted.
 id: 96 and 28 records inserted.
 id: 252 and 29 records inserted.
 id: 253 and 30 records inserted.
 id: 254 and 31 r

In [13]:
keyspace = ASTRA_DB_KEYSPACE
table = "ptc_inventory"
keyword = "laptop with 8gb"
embedding = generate_embedding(keyword)

cmd_select_products = f"""
    SELECT product_id, item_sku, item_name, product_url
    FROM {keyspace}.{table}
    ORDER BY text_embedding ANN OF {embedding}
    LIMIT 5 """

results = s.execute(cmd_select_products)

products = results._current_rows
for row in products:
  print(f"""{str(row.product_id)}, {row.item_sku}, {row.item_name}, {row.product_url}\n""")

444, LO-479, NB ThinkPad T440P i5-4200M-8GB-1TB-14-DW-1GB-Win8Pro64 (1Y) 20AW-S1MU00, https://ptc-computer.com.kh/product/NB-ThinkPad-T440P-i5-4200M-8GB-1TB-14-DW-1GB-Win8Pro64-1Y-20AW-S1MU00-bcmjr

442, LO-477, NB ThinkPad T440P i5-4200M-8GB-1TB-14-DW-1GB-Dos (1Y) 20AW-S1MS00, https://ptc-computer.com.kh/product/NB-ThinkPad-T440P-i5-4200M-8GB-1TB-14-DW-1GB-Dos-1Y-20AW-S1MS00-luRgH

1115, LB-201, NB VAIO S11 Silver i7-8550U-8GB-256GB-11.6FHD- Win10 Pro (NP11V1AV008P)(6M Part,3M battery), https://ptc-computer.com.kh/product/nb-vaio-s11-silver-i7-8550u-8gb-256gb-116fhd--win10-pro-np11v1av008p6m-part3m-battery-agpnb

319, LLE14-0709, Lenovo ThinkPad E14 Gen 4 Black i7-1355U-8GB-512GB SSD M.2-14"FHD-WiFi6-65W-BLKB-FP-RJ45-Dos(1Y), https://ptc-computer.com.kh/product/nb-lenovo-thinkpad-e14-gen-4-black-i5-1235u-8gb-512gb-ssd-m2-14fhd-wifi6-65w-blkb-fp-rj45-dos1y-cqugw

1116, LB-203, NB VAIO S11 White i7-8550U-8GB-256GB-11.6FHD- Win10 Pro (NP11V1AV010P)(6M Part,3M battery), https://ptc-comput

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
